#  aitextgen — Train a GPT-2 Text-Generating Model w/ GPU

by [Max Woolf](https://minimaxir.com)

*Last updated: Jul 5th, 2020*

Retrain an advanced text generating neural network on any text dataset **for free on a GPU using Colaboratory** using `aitextgen`!

For more about `aitextgen`, you can visit [this GitHub repository](https://github.com/minimaxir/aitextgen) or [read the documentation](https://docs.aitextgen.io/).


To get started:

1. Copy this notebook to your Google Drive to keep it and save your changes. (File -> Save a Copy in Drive)
2. Run the cells below:
*** 이전 사용 코드: https://github.com/YeonwooSung/KoGPT2_Lyricist/tree/master/src

In [ ]:
# !pip install -q aitextgen

!pip install -q transformers>=4.0.0
!pip install -q fire>=0.3.0
!pip install -q pytorch-lightning>=1.0.8
!pip install -q tokenizers>=1.0.0
!pip install -q torch>=1.6.0
!pip install -q aitextgen --no-deps

import logging
logging.basicConfig(
        format="%(asctime)s — %(levelname)s — %(name)s — %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO
    )

from aitextgen import aitextgen
from aitextgen.colab import mount_gdrive, copy_file_from_gdrive

     |████████████████████████████████| 573kB 8.1MB/s 


## GPU

Colaboratory uses a Nvidia P4, an Nvidia T4, or an Nvidia P100 GPU. For finetuning GPT-2 124M, any of these GPUs will be fine, but for text generation, a T4 or a P100 is ideal since they have more VRAM.

You can verify which GPU is active by running the cell below. If you want to try for a different GPU, go to **Runtime -> Factory Reset Runtime**.

In [ ]:
!nvidia-smi

Thu Feb  4 03:54:08 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   61C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Loading GPT-2

If you're retraining a model on new text, you need to download and load the GPT-2 model into the GPU. 

There are several sizes of GPT-2: currently, aitextgen only works with the smallest one:

* `124M` (default): the "small" model, 500MB on disk.

The next cell downloads it from Google's servers and saves it in the Colaboratory VM. If the model has already been downloaded, running this cell will reload it.

In [ ]:
ai = aitextgen(tf_gpt2="124M", to_gpu=True)

02/04/2021 03:54:08 — INFO — aitextgen — Downloading the 124M GPT-2 TensorFlow weights/config from Google's servers


02/04/2021 03:54:13 — INFO — aitextgen — Converting the 124M GPT-2 TensorFlow weights to PyTorch.


Converting TensorFlow checkpoint from /content/aitextgen/124M
Loading TF weight model/h0/attn/c_attn/b with shape [2304]
Loading TF weight model/h0/attn/c_attn/w with shape [1, 768, 2304]
Loading TF weight model/h0/attn/c_proj/b with shape [768]
Loading TF weight model/h0/attn/c_proj/w with shape [1, 768, 768]
Loading TF weight model/h0/ln_1/b with shape [768]
Loading TF weight model/h0/ln_1/g with shape [768]
Loading TF weight model/h0/ln_2/b with shape [768]
Loading TF weight model/h0/ln_2/g with shape [768]
Loading TF weight model/h0/mlp/c_fc/b with shape [3072]
Loading TF weight model/h0/mlp/c_fc/w with shape [1, 768, 3072]
Loading TF weight model/h0/mlp/c_proj/b with shape [768]
Loading TF weight model/h0/mlp/c_proj/w with shape [1, 3072, 768]
Loading TF weight model/h1/attn/c_attn/b with shape [2304]
Loading TF weight model/h1/attn/c_attn/w with shape [1, 768, 2304]
Loading TF weight model/h1/attn/c_proj/b with shape [768]
Loading TF weight model/h1/attn/c_proj/w with shape [1, 7

Save PyTorch model to aitextgen/pytorch_model.bin


02/04/2021 03:54:19 — INFO — aitextgen — Loading 124M GPT-2 model from /aitextgen.


Save configuration file to aitextgen/config.json


02/04/2021 03:54:24 — INFO — aitextgen — Using the default GPT-2 Tokenizer.


## Mounting Google Drive

The best way to get input text to-be-trained into the Colaboratory VM, and to get the trained model *out* of Colaboratory, is to route it through Google Drive *first*.

Running this cell (which will only work in Colaboratory) will mount your personal Google Drive in the VM, which later cells can use to get data in/out. (it will ask for an auth code; that auth is not saved anywhere)

In [ ]:
mount_gdrive()

Mounted at /content/drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Uploading a Text File to be Trained to Colaboratory

In the Colaboratory Notebook sidebar on the left of the screen, select *Files*. From there you can upload files:

![alt text](https://i.imgur.com/w3wvHhR.png)

Upload **any smaller text file** (for example, [a text file of Shakespeare plays](https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt)) and update the file name in the cell below, then run the cell.

In [ ]:
# file_name = "input.txt"
# file_name = "/content/drive/MyDrive/작사작곡_정리/작사/data/lyric_gpt2_518KB.txt"

If your text file is large (>10MB), it is recommended to upload that file to Google Drive first, then copy that file from Google Drive to the Colaboratory VM.

Additionally, you may want to consider [compressing the dataset to a cache first](https://docs.aitextgen.io/dataset/) on your local computer, then uploading the resulting `dataset_cache.tar.gz` and setting the `file_name`in the previous cell to that.

In [ ]:
# copy_file_from_gdrive(file_name)

## Finetune GPT-2

The next cell will start the actual finetuning of GPT-2 in aitextgen. It runs for `num_steps`, and a progress bar will appear to show training progress, current loss (the lower the better the model), and average loss (to give a sense on loss trajectory).

The model will be saved every `save_every` steps in `trained_model` by default, and when training completes. If you mounted your Google Drive, the model will _also_ be saved there in a unique folder.

The training might time out after 4ish hours; if you did not mount to Google Drive, make sure you end training and save the results so you don't lose them! (if this happens frequently, you may want to consider using [Colab Pro](https://colab.research.google.com/signup))

Important parameters for `train()`:

- **`line_by_line`**: Set this to `True` if the input text file is a single-column CSV, with one record per row. aitextgen will automatically process it optimally.
- **`from_cache`**: If you compressed your dataset locally (as noted in the previous section) and are using that cache file, set this to `True`.
- **`num_steps`**: Number of steps to train the model for.
- **`generate_every`**: Interval of steps to generate example text from the model; good for qualitatively validating training.
- **`save_every`**: Interval of steps to save the model: the model will be saved in the VM to `/trained_model`.
- **`save_gdrive`**: Set this to `True` to copy the model to a unique folder in your Google Drive, if you have mounted it in the earlier cells

Here are other important parameters for `train()` that are useful but you likely do not need to change.

- **`learning_rate`**: Learning rate of the model training.
- **`batch_size`**: Batch size of the model training; setting it too high will cause the GPU to go OOM.

In [ ]:
# ai.train(file_name,
#          line_by_line=False,
#          from_cache=False,
#          num_steps=5000,
#          generate_every=1000,
#          save_every=1000,
#          save_gdrive=False,
#          learning_rate=1e-4,
#          batch_size=1, 
#          )

You're done! Feel free to go to the **Generate Text From The Trained Model** section to generate text based on your retrained model.


## Load a Trained Model

Running the next cell will copy the `pytorch_model.bin` and the `config.json`file from the specified folder in Google Drive into the Colaboratory VM. (If no `from_folder` is specified, it assumes the two files are located at the root level of your Google Drive)

In [ ]:
# from_folder = None

# for file in ["pytorch_model.bin", "config.json"]:
#   if from_folder:
#     copy_file_from_gdrive(file, from_folder)
#   else:
#     copy_file_from_gdrive(file)

The next cell will allow you to load the retrained model + metadata necessary to generate text.

In [ ]:
# ai = aitextgen(model="pytorch_model.bin", config="config.json", to_gpu=True)
pyt_model = "/content/drive/MyDrive/작사작곡_정리/작사/result/20210114/pytorch_model.bin"
conf = "/content/drive/MyDrive/작사작곡_정리/작사/result/20210114/config.json"
ai = aitextgen(model=pyt_model, config=conf, to_gpu=True)

02/04/2021 03:55:02 — INFO — aitextgen — Loading GPT-2 model from provided /content/drive/MyDrive/작사작곡_정리/작사/result/20210114/pytorch_model.bin.
02/04/2021 03:55:14 — INFO — aitextgen — Using the default GPT-2 Tokenizer.


## Generate Text From The Trained Model

After you've trained the model or loaded a retrained model from checkpoint, you can now generate text. `generate()` without any parameters generates a single text from the loaded model to the console.

In [ ]:
ai.generate()

�에 앞으로 가자
우리들은 씩씩한 어린이라네
금수강산 이어받을 새싹이라네
하나 둘 셋 넷 앞으로 가자
두 주먹을 굳게 쥐고 앞으로 가자
우리들은 용감한 어린이라네
자유대한 길이 빛낼 새싹이라네
새들 노랫


If you're creating an API based on your model and need to pass the generated text elsewhere, you can do `text = ai.generate_one()`

You can also pass in a `prompt` to the generate function to force the text to start with a given character sequence and generate text from there (good if you add an indicator when the text starts).

You can also generate multiple texts at a time by specifing `n`. You can pass a `batch_size` to generate multiple samples in parallel, giving a massive speedup (in Colaboratory, set a maximum of 50 for `batch_size` to avoid going OOM).

Other optional-but-helpful parameters for `ai.generate()` and friends:

*  **`max_length`**: Number of tokens to generate (default 256, you can generate up to 1024 tokens with GPT-2, but it will be _much_ slower)
* **`temperature`**: The higher the temperature, the crazier the text (default 0.7, recommended to keep between 0.7 and 1.0)
* **`top_k`**: Limits the generated guesses to the top *k* guesses (default 0 which disables the behavior; if the generated output is super crazy, you may want to set `top_k=40`)
* **`top_p`**: Nucleus sampling: limits the generated guesses to a cumulative probability. (gets good results on a dataset with `top_p=0.9`)

In [ ]:
ai.generate(n=10,
            batch_size=5,
            prompt="동생과 함께 들판에 가서 강아지와 뛰어 놀면서 예쁜 꽃을 관찰한다. \n개울가에서 송사리 잡다가 물놀이하느라 엄마가 찾는 소리에  집으로 가는 이야기",
            max_length=300,
            temperature=1.0,
            top_p=1.0)

동생과 함께 들판에 가서 강아지와 뛰어 놀면서 예쁜 꽃을 관찰한다. 
개울가에서 송사리 잡다가 물놀이하느라 엄마가 찾는 소리에  집으로 가는 이야기 소리
밤 하늘 별빛 반짝일때면 문득 생각이 나요
내가 쌓은 속에 파란 속에서 느껴져요
매일 아침 �
동생과 함께 들판에 가서 강아지와 뛰어 놀면서 예쁜 꽃을 관찰한다. 
개울가에서 송사리 잡다가 물놀이하느라 엄마가 찾는 소리에  집으로 가는 이야기들
비단구두 사각사각 왔다가 반짝 꿈방울 속삭이면 수평선에 반짝이는 세상을 만들고 하늘까지 닿게
동생과 함께 들판에 가서 강아지와 뛰어 놀면서 예쁜 꽃을 관찰한다. 
개울가에서 송사리 잡다가 물놀이하느라 엄마가 찾는 소리에  집으로 가는 이야기 나누자.
빨간 꽃씨가 엄마가 웃음꽃이 활짝 웃는다고 잘잘 내게 춤춘다
비가 오면 떨어집니다 비가 �
동생과 함께 들판에 가서 강아지와 뛰어 놀면서 예쁜 꽃을 관찰한다. 
개울가에서 송사리 잡다가 물놀이하느라 엄마가 찾는 소리에  집으로 가는 이야기 소리
물결 넘어 앞에 내가 제일 좋아하는 우리 어린이
“어떻게 하면 어른이 될 수 있나요? 빨리 어린이�
동생과 함께 들판에 가서 강아지와 뛰어 놀면서 예쁜 꽃을 관찰한다. 
개울가에서 송사리 잡다가 물놀이하느라 엄마가 찾는 소리에  집으로 가는 이야기 소리
고요한 밤 거룩한 밤 어둠에 묻힌 밤
주의 부모 앉아서 감사 기도 드릴 때
아기 잘도 잔다 아기 �
동생과 함께 들판에 가서 강아지와 뛰어 놀면서 예쁜 꽃을 관찰한다. 
개울가에서 송사리 잡다가 물놀이하느라 엄마가 찾는 소리에  집으로 가는 이야기를 들었지
동화속 나라 공잡했던 밤새 웃음소리 맴을도 하늘 보면 파란 산새들이 나뭇잎 손뼉치며 �
동생과 함께 들판에 가서 강아지와 뛰어 놀면서 예쁜 꽃을 관찰한다. 
개울가에서 송사리 잡다가 물놀이하느라 엄마가 찾는 소리에  집으로 가는 이야기 들려줍니다

산도집도 아지는 산새소리 쪼로롱 꿈속에서 들려오고
쪼롱쪼로롱 산새소리에 산
동생과 함께 들판에 가서 강아지와 뛰어 놀면서 예쁜 꽃을 관

For bulk generation, you can generate a large amount of texts to a file and sort out the samples locally on your computer. The next cell will generate `num_files` files, each with `n` texts and whatever other parameters you would pass to `generate()`. The files can then be downloaded from the Files sidebar!

You can rerun the cells as many times as you want for even more generated texts!

In [ ]:
# num_files = 5

# for _ in range(num_files):
#   ai.generate_to_file(n=1000,
#                      batch_size=50,
#                      prompt="ROMEO:",
#                      max_length=256,
#                      temperature=1.0,
#                      top_p=0.9)

# LICENSE

MIT License

Copyright (c) 2020 Max Woolf

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.